In [1]:
from gurobipy import *
import pandas as pd
import numpy as np
import requests
import math
from io import StringIO
from sklearn.preprocessing import MinMaxScaler
from collections import namedtuple
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn import tree

In [2]:
def k_fold_split(df, k, random_state=1):
    split = {}
    df_shuffle = df.sample(frac=1, random_state=random_state)
    fold_index = np.array_split(df_shuffle.index, k)

    for i in range(k):
        fold = list(range(k))
        fold.remove(i) # remove i out for test set
        
        df_test = df.loc[fold_index[i]]
        df_train = pd.concat([df.loc[fold_index[i]] for i in fold])

        split[i] = {}
        split[i]['train'] = df_train
        split[i]['test'] = df_test
    
    return split

In [38]:
def oct_function(X_train, Y_train, max_depth, N_min = 1, alpha = 1e-10, C=1e+10, warmstrat_rules=None,  time_limit=600): 
    
    # Normalize Training set
    scaler = MinMaxScaler()
    X_train_scale = scaler.fit_transform(X_train)
    rows, cols = X_train.shape
    
    n_nodes = 2**(max_depth+1) - 1  # max_depth ชั้นถัดจาก root nodes, 7 nodes (D = 2) 
    K = len(Y_train.unique())  # K labels
    p = cols  # p features
    n = rows  # n observations
    alpha = alpha  # alpha (complexity)
    N_min = N_min  # min sample each leaf
    
    # node index
    branch_nodes = range(1, n_nodes//2+1) # index 1 to 3 
    leaf_nodes = range(n_nodes//2 + 1, n_nodes+1) # index 4 to 7

    # Epsilon
    eps_j = []
    for i in range(p):
        x = np.sort(X_train_scale[:,i])[::-1]
        dist_list = []
        for i in range(len(x)-1):
            dist = x[i]-x[i+1]
            if dist != 0:
                dist_list.append(dist)
        min_dist = np.min(dist_list)
        eps_j.append(min_dist)
    eps_max = np.max(eps_j)

    # Matrix Y
    Y_ik = -(np.ones([rows, K], dtype = int))
    Y_ik[Y_train.index, Y_train] = 1
    
    m = Model('mip1')
    
    m.Params.timelimit = time_limit   
    
    # baseline accuracy - L_hat loss
    L_hat = len(df['y']) - max(df['y'].value_counts())
    
    # ------------------------- Decision Variable ----------------------- #
    L_t = m.addVars(leaf_nodes, vtype = GRB.INTEGER, name = 'L_t')
    N_kt = m.addVars(K, leaf_nodes, vtype = GRB.INTEGER, name ='N_kt')
    N_t = m.addVars(leaf_nodes, vtype = GRB.INTEGER, name = 'N_t')
    c_kt = m.addVars(K, leaf_nodes, vtype = GRB.BINARY, name = 'c_kt')
    a_jt = m.addVars(p, branch_nodes, vtype = GRB.BINARY, name = 'a_jt')
    z_it = m.addVars(n, leaf_nodes, vtype = GRB.BINARY, name = 'z_it')
    l_t = m.addVars(leaf_nodes, vtype = GRB.BINARY, name = 'l_t')
    d_t = m.addVars(branch_nodes, vtype = GRB.BINARY, name = 'd_t')
    b_t = m.addVars(branch_nodes, vtype = GRB.CONTINUOUS, name = 'b_t')
    
    # Objective
    m.setObjective(((L_t.sum())/L_hat) + (alpha*(d_t.sum())), GRB.MINIMIZE)
    
    # ---------- C for tuning alpha -------------
    m.addConstr(d_t.sum() <= C)
    # -----------------------------------------------
    
    for i in leaf_nodes: 
        for j in range(K):
            m.addConstr(L_t[i] >= N_t[i] - N_kt[j,i] - n*(1-c_kt[j,i]))   # 1 /
        
    for i in leaf_nodes: 
        for j in range(K):
            m.addConstr(L_t[i] <= N_t[i] - N_kt[j,i] + n*c_kt[j,i])       # 2 /

    for i in leaf_nodes:
        m.addConstr(L_t[i] >= 0)  # 3 /

    for i in leaf_nodes: 
        for j in range(K):
            m.addConstr(N_kt[j,i] == 1/2 * sum((1 + Y_ik[:,j]) * z_it.select('*',i)))   # 4

    for i in leaf_nodes:
        m.addConstr(N_t[i] == z_it.sum('*',i))  # 5 /

    for i in leaf_nodes:
        m.addConstr(c_kt.sum('*',i) == l_t[i])   # 6 /  

    for i in range(n):
        m.addConstr(z_it.sum(i,'*') == 1)   # 9 /

    for i in leaf_nodes:
        for j in range(n):
            m.addConstr(z_it[j,i] <= l_t[i]) # 10 /

    for i in leaf_nodes:
        m.addConstr(z_it.sum('*', i) >= N_min * l_t[i]) # 11 /  

    for i in branch_nodes: 
        m.addConstr(a_jt.sum('*',i) == d_t[i]) # 12 /

    for i in branch_nodes:
        m.addConstr(b_t[i] <= d_t[i])  # 13 /

    for i in branch_nodes:
        if i != 1:
            m.addConstr(d_t[i] <= d_t[i//2]) # 14 /

    # Ancestor node
    parant_dict = {}
    for i in range(n_nodes):
        parant_dict[i+1] = (i+1)//2
    del parant_dict[1]
    print(parant_dict)

    ancestors_L = {}
    ancestors_R = {}
    for node in range(2,n_nodes+1):
        if node%2 == 0:
            if parant_dict[node] == 1:
                ancestors_L[node] = [1]
                ancestors_R[node] = []
            else:   
                ancestors_L[node] = ancestors_L[parant_dict[node]] + [parant_dict[node]]
                ancestors_R[node] = ancestors_R[parant_dict[node]]
        else:
            if parant_dict[node] == 1:
                ancestors_L[node] = []
                ancestors_R[node] = [1]
            else:
                ancestors_L[node] = ancestors_L[parant_dict[node]]
                ancestors_R[node] = ancestors_R[parant_dict[node]] + [parant_dict[node]]
    print(ancestors_L)
    print(ancestors_R)
    
    for i in range(n):
        for k in leaf_nodes:
            for a in ancestors_R[k]:
                m.addConstr(sum(a_jt.select('*',a)*X_train_scale[i,:]) >= b_t[a] - (1 - z_it[i,k]))  # 7

    for i in range(n):
        for k in leaf_nodes:
            for a in ancestors_L[k]:
                m.addConstr(sum(a_jt.select('*',a)*(X_train_scale[i,:] + eps_j)) <= b_t[a] + (1 + eps_max)*(d_t[a] - z_it[i,k])) # 8
    
    # --------------------------------------- WARM START -------------------------------------

#     max_depth_clf = max_depth - 1
#     n_nodes_clf = 2**(max_depth_clf+1) - 1
#     branch_node_clf = range(1, n_nodes_clf//2+1) 
#     leaf_nodes_clf = range(n_nodes_clf//2 + 1, n_nodes_clf+1) 
    
    if rules != None:
        for t in branch_nodes:
            # not split
            if rules[t].feat is None or rules[t].feat == tree._tree.TREE_UNDEFINED:
                d_t[t].start = 0
                for f in range(p):
                    a_jt[f,t].start = 0
            # split
            else:
                d_t[t].start = 1
                for f in range(p):
                    if f == int(rules[t].feat):
                        a_jt[f,t].start = 1
                    else:
                        a_jt[f,t].start = 0
#                 b_t[t].start = rules[t].threshold ## **

        # fix leaf nodes
        for t in leaf_nodes:
            # terminate early
            if rules[t].value is None:
                l_t[t].start = int(t % 2)
                # flows go to right
                if t % 2:
                    t_leaf = t
                    while rules[t].value is None:
                        t //= 2
                    for k in [0,1]:
                        if k == np.argmax(rules[t].value):
                            c_kt[k, t_leaf].start = 1
                        else:
                            c_kt[k, t_leaf].start = 0
                # nothing in left
                else:
                    for k in [0,1]:
                        c_kt[k, t].start = 0

            # terminate at leaf node
            else:
                l_t[t].start = 1
                for k in [0,1]:
                    if k == np.argmax(rules[t].value):
                        c_kt[k, t].start = 1
                    else:
                        c_kt[k, t].start = 0

    # ------------------------------------------- End WARM START -----------------------------------------------

    # ---------------- Extended model handle missing, manually add feature index -----------------------
    
    for a, b in zip([62],[0]): # a: feature with missing, b: missing flag
        for i in range(1, (n_nodes//4)+1): # feature A split หลัง flag missing feature A
            m.addConstr(a_jt[a, i*2] + a_jt[a, (i*2)+1] == a_jt[b, i])
    
    for i in [62]:
        m.addConstr(a_jt[i, 1] == 0)
    # -------------------------------------------------------------------------------------------------------------------
    
    m.update()
    m.optimize()
    
    variable_result = {'L_t':L_t, 'N_kt':N_kt, 'N_t':N_t, 'c_kt':c_kt, 'a_jt':a_jt, 'z_it':z_it, 'l_t':l_t, 'd_t':d_t, 'b_t':b_t}
    
    return m, branch_nodes, leaf_nodes, variable_result

In [4]:
def spliting_rules_function(m, X_train):
    spliting = {}
    # ดึง VarName เอาเลข node
    # round เพราะ a_jt, c_kt บางครั้งมันเป็น 0.999.. 
    a_jt = [var.VarName for var in m.getVars() if "a_jt" in var.VarName and round(var.x) == 1]  
    for i in a_jt:
        spliting[int(re.findall("[0-9]+",i)[1])] = {}
        spliting[int(re.findall("[0-9]+",i)[1])]['feature'] = int(re.findall("[0-9]+",i)[0])

    b_t = [var for var in m.getVars() if "b_t" in var.VarName]
    for i in b_t:
        if int(re.findall("[0-9]+",i.VarName)[0]) in spliting.keys():
            spliting[int(re.findall("[0-9]+",i.VarName)[0])]['value'] = i.x
    for i in spliting:
        spliting[i]['value_denorm'] = round((spliting[i]['value'] * (X_train.iloc[:,spliting[i]['feature']].max()  - X_train.iloc[:,spliting[i]['feature']].min())) \
                                                                                + X_train.iloc[:,spliting[i]['feature']].min())
         
    return spliting

In [5]:
## label map 2, new ver.
## ดูว่าแต่ละ obs อยู่ leaf node ไหนจาก z_it
def label_with_theshold_2(z_it, Y_train, theshold=0.5):
    tmp = [i for i in z_it if z_it[i].x == 1]
    tmp_df = pd.DataFrame(tmp,columns=['obs','leaf_node'])
    tmp_df['actual_label'] = Y_train
    tmp_df = tmp_df.groupby('leaf_node').agg({'actual_label':['count','sum']})
    tmp_df['predict_prob'] = tmp_df['actual_label']['sum'] / tmp_df['actual_label']['count']
    tmp_df['predict_label'] = np.where(tmp_df['predict_prob']>=theshold, 1, 0)
    
    return tmp_df['predict_label'].to_dict()

# label_with_theshold_2(variable_result['z_it'], Y_train, theshold=0.5)

In [6]:
def predict_function(spliting, label_map, X):
    X = np.array(X)
    d_t = variable_result['d_t']  # variable result from oct
    pred_leaf = []
    for i in range(len(X)):
        t = 1
        while t not in leaf_nodes:
            if d_t[t].x != 0:
                if X[i][spliting[t]['feature']] < spliting[t]['value_denorm']:   # *** 
                    t = t*2     # left node
                else:
                    t = (t*2) + 1     # right node
            else:
                t = (t*2) + 1     # not split (d_t[t]=0) go to right node
        pred_leaf.append(t)
    pred_label = [label_map[i] for i in pred_leaf]
    
    return pred_leaf, pred_label

In [7]:
## Evaluation AUC
def auc_oct(X, Y):
    theshold_list = []
    tpr_list = []
    fpr_list = []
    
    for i in np.array(list(range(0,101,1)))/100:
        
        label_map = label_with_theshold_2(variable_result['z_it'], Y_train, theshold=i)
        
        predict_leaf, predict_label = predict_function(spliting_rules, label_map, X)  # predict Y
        
        predict_label = pd.Series(predict_label)

        tp = sum(predict_label[predict_label==1] == Y[predict_label[predict_label==1].index]) # actual Y
        fp = sum(predict_label[predict_label==1] != Y[predict_label[predict_label==1].index])
        tn = sum(predict_label[predict_label==0] == Y[predict_label[predict_label==0].index])
        fn = sum(predict_label[predict_label==0] != Y[predict_label[predict_label==0].index])

        if tp == 0:
            tpr = 0
        else:
            tpr = tp/(tp+fn)

        if fp == 0:
            fpr = 0
        else:
            fpr = fp/(fp+tn)

        theshold_list.append(i)
        tpr_list.append(tpr)
        fpr_list.append(fpr)

    from sklearn import metrics
    auc = metrics.auc(np.array(fpr_list), np.array(tpr_list))
    
    return auc

In [8]:
## Warm Start
def cart_with_n_split(X_train, Y_train, max_depth, min_samples_leaf, C):
#     scaler = MinMaxScaler()
#     X_train_scale = scaler.fit_transform(X_train)
    
    clf_b = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=max_depth, min_samples_leaf=min_samples_leaf) 
    path = clf_b.cost_complexity_pruning_path(X_train, Y_train)
    ccp_alphas, impurities = path.ccp_alphas, path.impurities

    for ccp_alpha in ccp_alphas:
        clf_b = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=max_depth, ccp_alpha=ccp_alpha, min_samples_leaf=min_samples_leaf)
        clf_b.fit(X_train, Y_train)
        number_split = (clf_b.tree_.node_count - 1)/ 2
        if number_split <= C : 
            ## ใช้ number_split ใกล้เคียง C ที่สุด และไม่เกิน C
            # จน.ครั้งที่ split (C) อาจจะมีไม่ครบทุกตัวเลข ขึ้นอยู่กับ data set/min_sample_leaf 
            # จน.ครั้งที่ split (C) เริ่มจาก มากไปน้อย - break when <=
            break
    return clf_b

# -------------------------------------------------------------------------------------------------------- #

def getRules(clf): # get spliting rules from CART
    max_depth = clf.max_depth
    n_nodes = 2**(max_depth+1) - 1
    branch_nodes = range(1, n_nodes//2+1) 
    leaf_nodes = range(n_nodes//2 + 1, n_nodes+1) 

    # node index map
    node_map = {1:0}
    for t in list(branch_nodes):
        # terminal
        node_map[2*t] = -1      
        node_map[2*t+1] = -1
        
        l = clf.tree_.children_left[node_map[t]]  # left
        node_map[2*t] = l
        
        r = clf.tree_.children_right[node_map[t]]  # right
        node_map[2*t+1] = r
        
    # rules
    rule = namedtuple('Rules', ('feat', 'threshold', 'value'))
    rules = {}
    
    for t in list(branch_nodes): # branch_node
        i = node_map[t]
        if i == -1:
            r = rule(None, None, None)
        else:
            r = rule(clf.tree_.feature[i], clf.tree_.threshold[i], clf.tree_.value[i,0])
        rules[t] = r 
        
    for t in list(leaf_nodes): # leaf_node
        i = node_map[t]
        if i == -1:
            r = rule(None, None, None)
        else:
            r = rule(None, None, clf.tree_.value[i,0])
        rules[t] = r

    return rules

-----

In [9]:
df = pd.read_csv('german.data', sep=' ', header=None)

df.columns = ['status_account', 'duration', 'credit_history', 'purpose', 'credit_amt', 'saving_account', 'present_employment_since', 'installment_rate'
                            , 'status_gender', 'other_debtors', 'present_residence_since', 'property', 'age', 'other_installment_plan', 'housing', 'n_creditcard'
                            , 'job', 'n_people_liable', 'telephone', 'foreign_worker', 'y']

numeric = ['duration', 'credit_amt', 'installment_rate', 'present_residence_since', 'age', 'n_creditcard', 'n_people_liable'
#           ,'correlate_feat'
          ]
category = ['status_account','credit_history', 'purpose', 'saving_account', 'present_employment_since'
                 ,'status_gender','other_debtors','property', 'other_installment_plan','housing', 'job','telephone', 'foreign_worker'
#                  ,'missing_flag'
           ]

df['y'] = np.where(df['y'] == 1, 0, 1)

In [39]:
df_2 = pd.concat([pd.get_dummies(df[category]), df[numeric], df['y']], axis=1)

df_2 = df_2.sample(frac=1, random_state=0) # ********

df_train = df_2[:750]
df_holdout = df_2[750:]

X_train = df_train.drop(['y'], axis=1)
Y_train = df_train['y']

X_test = df_holdout.drop(['y'], axis=1)
Y_test = df_holdout['y']

X_train.reset_index(drop=True, inplace=True)
Y_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)

# Model Part

max_depth = 2
C = 7

## CART rules to set warmstar
clf_b = cart_with_n_split(X_train, Y_train, max_depth = max_depth, min_samples_leaf=int(len(X_train)*0.05), C = C)
rules = getRules(clf_b)

## OCT
m, branch_nodes, leaf_nodes, variable_result = oct_function(X_train, Y_train
                                                                                        , max_depth=max_depth
#                                                                                         ,N_min=len(X_train)*0.05   # ******
                                                                                        , N_min = 1
                                                                                        , C=C
                                                                                        , warmstrat_rules=None
                                                                                        , time_limit=300)

spliting_rules = spliting_rules_function(m, X_train)

label_map = label_with_theshold_2(variable_result['z_it'], Y_train, theshold=0.5)

predict_leaf_train, predict_label_train = predict_function(spliting_rules, label_map, X_train)  
predict_leaf_test, predict_label_test = predict_function(spliting_rules, label_map, X_test)

Set parameter TimeLimit to value 300
{2: 1, 3: 1, 4: 2, 5: 2, 6: 3, 7: 3}
{2: [1], 3: [], 4: [1, 2], 5: [1], 6: [3], 7: []}
{2: [], 3: [1], 4: [], 5: [2], 6: [1], 7: [1, 3]}


In [21]:
spliting_rules

{1: {'feature': 3, 'value': 1.0, 'value_denorm': 1},
 2: {'feature': 54, 'value': 0.4821428571428572, 'value_denorm': 31},
 3: {'feature': 55, 'value': 0.590073731704633, 'value_denorm': 10974}}

In [26]:
label_map

{4: 0, 5: 1, 6: 0, 7: 1}

In [29]:
X_test.iloc[:, [3,54,55]] 

,status_account_A14,duration,credit_amt
0,0,12,4843
1,1,18,3780
2,0,12,2028
3,1,24,2284
4,0,15,2728
...,...,...,...
245,0,12,1082
246,0,27,3915
247,1,9,3832
248,0,18,1928


In [27]:
X_test

,status_account_A11,status_account_A12,status_account_A13,status_account_A14,credit_history_A30,credit_history_A31,credit_history_A32,credit_history_A33,credit_history_A34,purpose_A40,...,telephone_A192,foreign_worker_A201,foreign_worker_A202,duration,credit_amt,installment_rate,present_residence_since,age,n_creditcard,n_people_liable
0,1,0,0,0,0,0,0,0,1,1,...,1,1,0,12,4843,3,4,43,2,1
1,0,0,0,1,0,0,0,0,1,0,...,1,1,0,18,3780,3,2,35,2,1
2,0,1,0,0,0,0,1,0,0,0,...,0,1,0,12,2028,4,2,30,1,1
3,0,0,0,1,0,0,1,0,0,0,...,1,1,0,24,2284,4,2,28,1,1
4,0,1,0,0,0,0,0,0,1,0,...,1,1,0,15,2728,4,2,35,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,1,0,0,0,1,0,0,0,0,1,...,0,1,0,12,1082,4,4,48,2,1
246,0,1,0,0,0,0,1,0,0,0,...,1,1,0,27,3915,4,2,36,1,2
247,0,0,0,1,0,0,1,0,0,0,...,0,1,0,9,3832,1,4,64,1,1
248,0,1,0,0,0,0,0,0,1,0,...,0,1,0,18,1928,2,2,31,2,1


In [42]:
for i in m.getVars():
#     print(i.VarName, i.start)
    print(i.VarName, i.X)

AttributeError: Unable to retrieve attribute 'X'

In [17]:
# Evaluation
# Accuracy - theshold from label_map 
print(" -----------------------------------------------------")
print(f"max_depth: {max_depth}     C: {C} ")
print(sum(Y_train == predict_label_train)/len(Y_train))
print(sum(Y_test == predict_label_test)/len(Y_test))

 -----------------------------------------------------
max_depth: 2     C: 7 
0.7333333333333333
0.704


In [18]:
## Spliting Rules

spliting_rules

{1: {'feature': 3, 'value': 1.0, 'value_denorm': 1},
 2: {'feature': 54, 'value': 0.4821428571428572, 'value_denorm': 31},
 3: {'feature': 55, 'value': 0.590073731704633, 'value_denorm': 10974}}

--------------------

## Extended model - handle missing example

In [44]:
df = pd.read_csv('german.data', sep=' ', header=None)

df.columns = ['status_account', 'duration', 'credit_history', 'purpose', 'credit_amt', 'saving_account', 'present_employment_since', 'installment_rate'
                            , 'status_gender', 'other_debtors', 'present_residence_since', 'property', 'age', 'other_installment_plan', 'housing', 'n_creditcard'
                            , 'job', 'n_people_liable', 'telephone', 'foreign_worker', 'y']

numeric = ['duration', 'credit_amt', 'installment_rate', 'present_residence_since', 'age', 'n_creditcard', 'n_people_liable'
          ,'correlate_feat'
          ]
category = ['status_account','credit_history', 'purpose', 'saving_account', 'present_employment_since'
                 ,'status_gender','other_debtors','property', 'other_installment_plan','housing', 'job','telephone', 'foreign_worker'
                 ,'missing_flag'
           ]

df['y'] = np.where(df['y'] == 1, 0, 1)

In [45]:
null_1 = np.empty((100,))
null_1[:100] = np.nan

rand_highvalue = np.random.randint(50,400,size=200)
value_1 = np.concatenate((rand_highvalue, null_1))

In [46]:
null_0 = np.empty((400,))
null_0[:400] = np.nan

rand_lowvalue = np.random.randint(0,55,size=300)
value_0 = np.concatenate((rand_lowvalue, null_0))

In [47]:
df_tmp1 = df[df['y']==1]
df_tmp1['correlate_feat'] = value_1

df_tmp0 = df[df['y']==0]
df_tmp0['correlate_feat'] = value_0

C:\Users\PONGTH~1\AppData\Local\Temp/ipykernel_13908/3330032030.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp1['correlate_feat'] = value_1
C:\Users\PONGTH~1\AppData\Local\Temp/ipykernel_13908/3330032030.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp0['correlate_feat'] = value_0


In [48]:
df_withmissing = pd.concat([df_tmp0,df_tmp1]).sort_index()

In [49]:
df_withmissing['correlate_feat'].fillna(-1,inplace=True)
df_withmissing['missing_flag'] = np.where(df_withmissing['correlate_feat']== -1, 1, 0)

In [51]:
df_withmissing_2 = pd.concat([pd.get_dummies(df_withmissing[category]), df_withmissing[numeric], df_withmissing['y']], axis=1)
df_withmissing_2 = df_withmissing_2.sample(frac=1, random_state=0) # ********

In [52]:
df_withmissing_2

,missing_flag,status_account_A11,status_account_A12,status_account_A13,status_account_A14,credit_history_A30,credit_history_A31,credit_history_A32,credit_history_A33,credit_history_A34,...,foreign_worker_A202,duration,credit_amt,installment_rate,present_residence_since,age,n_creditcard,n_people_liable,correlate_feat,y
993,1,1,0,0,0,0,0,1,0,0,...,0,36,3959,4,3,30,1,1,-1.0,0
859,1,0,0,0,1,0,0,1,0,0,...,1,9,3577,1,2,26,1,2,-1.0,0
298,0,0,0,0,1,0,0,1,0,0,...,0,18,2515,3,4,43,1,1,10.0,0
553,1,0,1,0,0,0,0,0,0,1,...,0,12,1995,4,1,27,1,1,-1.0,0
672,1,0,0,0,1,0,0,1,0,0,...,0,60,10366,2,4,42,1,1,-1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,1,1,0,0,0,1,0,0,0,0,...,0,12,1082,4,4,48,2,1,-1.0,1
192,0,0,1,0,0,0,0,1,0,0,...,0,27,3915,4,2,36,1,2,249.0,1
629,1,0,0,0,1,0,0,1,0,0,...,0,9,3832,1,4,64,1,1,-1.0,0
559,0,0,1,0,0,0,0,0,0,1,...,0,18,1928,2,2,31,2,1,138.0,1


In [20]:
# df_2 = pd.concat([pd.get_dummies(df[category]), df[numeric], df['y']], axis=1)
# df_2 = df_2.sample(frac=1, random_state=0) # ********

# df_train = df_2[:750]
# df_holdout = df_2[750:]

# X_train = df_train.drop(['y'], axis=1)
# Y_train = df_train['y']

# X_test = df_holdout.drop(['y'], axis=1)
# Y_test = df_holdout['y']

# X_train.reset_index(drop=True, inplace=True)
# Y_train.reset_index(drop=True, inplace=True)
# X_test.reset_index(drop=True, inplace=True)
# Y_test.reset_index(drop=True, inplace=True)

X_train = df_withmissing_2.drop(['y'], axis=1)
Y_train = df_withmissing_2['y']

X_train.reset_index(drop=True,inplace=True)
Y_train.reset_index(drop=True,inplace=True)

max_depth = 3
C = 7

## CART rules to set warmstart
clf_b = cart_with_n_split(X_train, Y_train, max_depth = max_depth
                          , min_samples_leaf=int(len(X_train)*0.05)
#                           ,min_samples_leaf = 1
                          , C = C)
rules = getRules(clf_b)

## OCT
m, branch_nodes, leaf_nodes, variable_result = oct_function(X_train, Y_train, max_depth=max_depth
                                                                                        ,N_min=len(X_train)*0.05   # ******
#                                                                                         ,N_min = 1
                                                                                        , C=C
                                                                                        , warmstrat_rules=None
                                                                                        , time_limit=7200)

spliting_rules = spliting_rules_function(m, X_train) 
label_map = label_with_theshold_2(variable_result['z_it'], Y_train, theshold=0.5)

predict_leaf_train, predict_label_train = predict_function(spliting_rules, label_map, X_train)  
# predict_leaf_test, predict_label_test = predict_function(spliting_rules, label_map, X_test)

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 33105 rows, 8511 columns and 1095647 nonzeros
Model fingerprint: 0xcfffd395
Variable types: 7 continuous, 8504 integer (8472 binary)
Coefficient statistics:
  Matrix range     [6e-05, 1e+03]
  Objective range  [1e-10, 3e-03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]

User MIP start did not produce a new incumbent solution
User MIP start violates constraint R33101 by 1.000000000

Presolve removed 31 rows and 16 columns (presolve time = 9s) ...
Presolve added 991 rows and 1006 columns
Presolve time: 9.83s
Presolved: 34096 rows, 9517 columns, 930667 nonzeros
Variable types: 1013 continuous, 8504 integer (8480 binary)
Found heuristic solution: objective 1.0000000

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0000000e-10  

 21614  5141    0.54333   55  698    0.92333    0.00000   100%   207 1446s
 21796  5170    0.58501   51  612    0.92333    0.00000   100%   208 1488s
H21817  5121                       0.8633333    0.00000   100%   208 1488s
 21885  5143     cutoff   58         0.86333    0.00000   100%   208 1497s
 22221  5145    0.55333   60  516    0.86333    0.00000   100%   207 1508s
 22462  5178    0.00667   47 1072    0.86333    0.00000   100%   209 1520s
 22646  5199    0.75000   62  271    0.86333    0.00000   100%   211 1541s
 22814  5223 infeasible   61         0.86333    0.00000   100%   212 1554s
 22929  5226    0.76690   61  199    0.86333    0.00000   100%   215 1715s
 22940  5223    0.83033   62   29    0.86333    0.00000   100%   216 1726s
 23066  5230    0.36333   50  310    0.86333    0.00000   100%   219 1750s
 23129  5241    0.00000   48  769    0.86333    0.00000   100%   221 1769s
 23211  5209 infeasible   57         0.86333    0.00000   100%   224 1783s
 23327  5254    0.73667  

 138121 17345 infeasible   47         0.37333    0.00000   100%   195 6162s
 138614 17403    0.00000  124 1432    0.37333    0.00000   100%   195 6412s
 139602 17316 infeasible  125         0.37333    0.00000   100%   195 6545s
H139617 16935                       0.3433333    0.00000   100%   195 6545s
 139961 17983    0.00000   53 1030    0.34333    0.00000   100%   197 6587s
 145697 17727    0.21667   75 1071    0.34333    0.00000   100%   195 6607s
 145978 17887 infeasible   63         0.34333    0.00000   100%   195 6626s
 148942 17592 infeasible   85         0.34333    0.00000   100%   195 6646s
 149251 17896    0.00000   41 1268    0.34333    0.00000   100%   195 6664s
 151799 17880    0.00000   85 1101    0.34333    0.00000   100%   194 6678s
 152217 18231 infeasible   92         0.34333    0.00000   100%   194 6692s
 153741 17982 infeasible  126         0.34333    0.00000   100%   194 6752s
 154021 19362    0.00000   48 1628    0.34333    0.00000   100%   195 6805s
 159757 1907

In [21]:
print(" -----------------------------------------------------")
print(f"max_depth: {max_depth}     C: {C} ")
print(sum(Y_train == predict_label_train)/len(Y_train))
# print(sum(Y_test == predict_label_test)/len(Y_test))
print(" ------------------------------------------------------- ")

## Spliting Rules
spliting_rules

 -----------------------------------------------------
max_depth: 3     C: 7 
0.897
 ------------------------------------------------------- 


{3: {'feature': 0, 'value': 1.0, 'value_denorm': 1},
 2: {'feature': 4, 'value': -0.0, 'value_denorm': 0},
 5: {'feature': 27, 'value': 0.0, 'value_denorm': 0},
 4: {'feature': 55, 'value': 0.0, 'value_denorm': 4},
 1: {'feature': 56, 'value': 0.0, 'value_denorm': 250},
 7: {'feature': 60, 'value': 0.33333333333333326, 'value_denorm': 2},
 6: {'feature': 62, 'value': 0.14141414141414144, 'value_denorm': 55}}

----------------